In [ ]:
!pip install -q tensorflow-cpu
!pip install -q -U keras-nlp tensorflow-hub
!pip install -q -U keras==3.1.1

In [ ]:
import os

#preallocate 100% of the total TPU memory to avoid memory fragmentation on the JAX backend
os.environ["KERAS_BACKEND"] = "jax"
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]="1.00"

In [ ]:
import keras
import keras_nlp

In [ ]:
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_instruct_2b_en")
gemma_lm.summary()

In [ ]:
#example
print(gemma_lm.generate(["How far is the Sun from earth", "What is the Sun made of?"], max_length=30))

# Finetuning step

In [ ]:
import os
import jax

os.environ["KERAS_BACKEND"] = "jax"
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "0.9"

jax.devices()

In [ ]:
import keras
import keras_nlp

gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_2b_en")

gemma_lm.summary()

In [ ]:
import pandas as pd

df = pd.read_csv('/kaggle/input/openhermes/openhermes.csv')
df.head()

In [ ]:
template = "Instruction:\n{instruction}\n\nResponse:\n{output}"
data = [template.format(**row) for index, row in df.iterrows()]

In [ ]:
data = data[:1000]
print(data[0])

In [ ]:
gemma_lm.backbone.enable_lora(rank=4)

In [ ]:
gemma_lm.preprocessor.sequence_length = 512

optimizer = keras.optimizers.AdamW(
    learning_rate=5e-5,
    weight_decay=0.01,
)

optimizer.exclude_from_weight_decay(var_names=["bias", "scale"])

gemma_lm.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=optimizer,
    weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

gemma_lm.summary()

In [ ]:
gemma_lm.fit(data, epochs=1, batch_size=1)

In [ ]:
gemma_lm.save_weights('gemma_2b_openhermes.weights.h5')

source: https://www.datacamp.com/tutorial/combine-google-gemma-with-tpus-fine-tune-and-run-inference-with-enhanced-performance-and-speed